
<h1 align=center><font size = 5>Segmenting and Clustering Neighborhoods in Toronto</font></h1>

### Github is not rendering/allowing all html tags probably for security reasons. Please check the Github link in https://nbviewer.jupyter.org/.

## Table of Contents

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>

1. <a href="#item1">Download and Explore Dataset</a>

2. <a href="#item2">Get latitude and the longitude coordinates</a>

3. <a href="#item3">Explore and cluster the neighborhoods</a>
 
</font>
</div>

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [131]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

<a id='item1'></a>

## 1. Download and Explore Dataset

We will download neighbourhood data for Toronto from the website https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M. We will start by getting the contents of the webpage into a variable.

In [132]:
html_str = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
result = requests.get(html_str)
result.status_code

200

Initialize a BeautifulSoup object with text from the webpage

In [133]:
soup = BeautifulSoup(result.text, 'html.parser')

Get the text from the first table in the webpage which contains the data we need.

In [134]:
table_text = soup.find_all('table')[0].get_text()

Create an empty Dataframe with the columns 'PostalCode', 'Borough' & 'Neighbourhood'

In [135]:
column_names = ['PostalCode', 'Borough', 'Neighbourhood'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,PostalCode,Borough,Neighbourhood


Populate the dataframe with the table text data from the webpage.

In [136]:
i = 0
for line in table_text.split('\n'):
    if(len(line.strip()) == 0):
        continue
    i = i + 1
    if(i == 1):
        postalCode = line.strip() 
    elif(i == 2):
        borough = line.strip()
    elif(i == 3):
        neighborhood = line.strip()
        neighborhoods = neighborhoods.append({'PostalCode': postalCode,
                                          'Borough': borough,
                                          'Neighbourhood': neighborhood}, ignore_index=True)
        i = 0
neighborhoods.head()

,PostalCode,Borough,Neighbourhood
0,Postcode,Borough,Neighbourhood
1,M1A,Not assigned,Not assigned
2,M2A,Not assigned,Not assigned
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village


Print the shape of the dataframe

In [137]:
neighborhoods.shape

(289, 3)

Remove the first row of the table which contains the name of the columns. We will also remove the rows with a borough that is Not assigned.

In [138]:
neighborhoods = neighborhoods[(neighborhoods['PostalCode'] != 'Postcode') & (neighborhoods['Borough'] != 'Not assigned')]
neighborhoods.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


Print the shape of the dataframe to check how many rows we have actually deleted.

In [139]:
neighborhoods.shape

(211, 3)

If a cell has a borough but a Not assigned neighborhood, then we will assign the borough name to the neighborhood.

In [140]:
neighborhoods['Neighbourhood'].replace('Not assigned', neighborhoods['Borough'], inplace = True)
neighborhoods.head()

,PostalCode,Borough,Neighbourhood
3,M3A,North York,Parkwoods
4,M4A,North York,Victoria Village
5,M5A,Downtown Toronto,Harbourfront
6,M5A,Downtown Toronto,Regent Park
7,M6A,North York,Lawrence Heights


In [141]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 211 neighborhoods.


We will check that the boroughs are same for the same 'PostalCode'.

In [142]:
def myFuncToCheckUniquenessOfBorough(df):
    uniqueCount = len(df['Borough'].unique())
    if(uniqueCount != 1):
        print(df['Borough'], " is not unique")
neighborhoods.groupby(['PostalCode']).apply(myFuncToCheckUniquenessOfBorough)

""


More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma for all such scenarios. Assigning the resultant dataframe to a new one.

In [143]:
def myFunc(df):
    newNeibourhood = str()
    for index, row in df.iterrows():
        newNeibourhood = newNeibourhood + "," + row[2]
    d = {"PostalCode" : [row[0]], "Borough" : [row[1]] , "Neighbourhood" : [newNeibourhood[1:]]}
    returnDf = pd.DataFrame(d)
    return returnDf

neighborhoods_grouped = neighborhoods.groupby(['PostalCode']).apply(myFunc)
neighborhoods_grouped.head()

,,PostalCode,Borough,Neighbourhood
PostalCode,,,,
M1B,0,M1B,Scarborough,"Rouge,Malvern"
M1C,0,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,0,M1E,Scarborough,"Guildwood,Morningside,West Hill"
M1G,0,M1G,Scarborough,Woburn
M1H,0,M1H,Scarborough,Cedarbrae


Dropping the colum named PostalCode in order to reindex the dataframe.

In [144]:
neighborhoods_grouped.drop(columns={'PostalCode'}, inplace = True)
neighborhoods_grouped.head()

,,Borough,Neighbourhood
PostalCode,,,
M1B,0,Scarborough,"Rouge,Malvern"
M1C,0,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,0,Scarborough,"Guildwood,Morningside,West Hill"
M1G,0,Scarborough,Woburn
M1H,0,Scarborough,Cedarbrae


Reindexing the dataframe

In [145]:
neighborhoods_grouped.reset_index(inplace = True)
neighborhoods_grouped.head()

,PostalCode,level_1,Borough,Neighbourhood
0,M1B,0,Scarborough,"Rouge,Malvern"
1,M1C,0,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,0,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,0,Scarborough,Woburn
4,M1H,0,Scarborough,Cedarbrae


Removing the column named 'level_1' resulted due to the reindexing.

In [146]:
neighborhoods_grouped.drop(columns={'level_1'}, inplace = True)

Printing first 20 elements of the dataframe.

In [147]:
neighborhoods_grouped.head(20)

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park"
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge"
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff,Cliffside West"


<a id='item2'></a>

## 2. Get latitude and the longitude coordinates

import the asscociated packages

In [148]:
from geopy.geocoders import Nominatim
import numpy as np
import geocoder

Checking the size of the dataframe

In [149]:
neighborhoods_grouped.shape

(103, 3)

In [150]:

# listOflats = list()
# listOflongs = list()
# geolocator = Nominatim(user_agent="toronto_explorer")

# for post in neighborhoods_grouped['PostalCode']:
#     address = '{}, Toronto, Ontario'.format(post)
#     location = None
#     while(location is None):
#         location = geolocator.geocode(address)
#     print(location.longitude)
#     listOflats.append(location.latitude)
#     listOflongs.append(location.longitude)

This code is not working. So reverting back to the csv file provided.

In [151]:
geospatialData = 'Geospatial_Coordinates.csv'
df_geospatialData = pd.read_csv(geospatialData)
df_geospatialData.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Assigning the dataframe to a new one

In [152]:
neighborhoods_LatLong = neighborhoods_grouped

Populating the new columns 'Latitude' and 'Longitude' in the new dataframe named neighborhoods_LatLong

In [153]:
neighborhoods_LatLong['Latitude'] = df_geospatialData['Latitude']
neighborhoods_LatLong['Longitude'] = df_geospatialData['Longitude']

Checking whether all the rows were populated correctly for Latitude

In [154]:
neighborhoods_LatLong[neighborhoods_LatLong['Latitude'] == None]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


Checking whether all the rows were populated correctly for Longitude

In [155]:
neighborhoods_LatLong[neighborhoods_LatLong['Longitude'] == None]

,PostalCode,Borough,Neighbourhood,Latitude,Longitude


Showing the result

In [156]:
neighborhoods_LatLong.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


<a id='item3'></a>

## 3. Explore and cluster the neighborhoods

Import necessary libraries

In [157]:
import folium

Get the latitude and longitude of Toronto

In [158]:
address = 'Toronto, Canada'

geolocator_toronto = Nominatim(user_agent="toronto_neighbourhood_explorer")
location_toronto = geolocator_toronto.geocode(address)
latitude_toronto = location_toronto.latitude
longitude_toronto = location_toronto.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude_toronto, longitude_toronto))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


#### Create a map of Toronto with neighborhoods superimposed on top.

In [159]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods_LatLong['Latitude'], neighborhoods_LatLong['Longitude'], neighborhoods_LatLong['Borough'], neighborhoods_LatLong['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [160]:
neighborhoods_with_Toronto_data = neighborhoods_LatLong[neighborhoods_LatLong['Borough'].str.contains('Toronto')].reset_index(drop=True)
neighborhoods_with_Toronto_data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


let's visualize the neighbourhoods with Toronto in their name.

In [161]:
# create map of Manhattan using latitude and longitude values
map_boroughs_with_Toronto = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# add markers to map
for lat, lng, label in zip(neighborhoods_with_Toronto_data['Latitude'], neighborhoods_with_Toronto_data['Longitude'], neighborhoods_with_Toronto_data['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boroughs_with_Toronto)  
    
map_boroughs_with_Toronto

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

#### Define Foursquare Credentials and Version

In [186]:
# @hidden_cell
CLIENT_ID = 'SBASIHEA4ADJICWW4MS2GWNQCRUPFIQ0X5F5D3LEWVJY0BGT' # your Foursquare ID
CLIENT_SECRET = 'BZCUOMGEO30UAYFW1WRUZJZCOGF0W43L0UGG414HKINUMVU0' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

### Explore all Neighborhoods with Toronto in their names

#### Let's create a function to get information for the neighbourhoods

In [163]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now we get information on each neighborhood and create a new dataframe called *neighborhoods_with_Toronto_venues*.

In [164]:
radius = 500
LIMIT = 100
neighborhoods_with_Toronto_venues = getNearbyVenues(names=neighborhoods_with_Toronto_data['Neighbourhood'],
                                                       latitudes=neighborhoods_with_Toronto_data['Latitude'],
                                                       longitudes=neighborhoods_with_Toronto_data['Longitude']
                                                   )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront,Regent Park
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvall

Check the size and data of the new dataframe

In [165]:
print(neighborhoods_with_Toronto_venues.shape)
neighborhoods_with_Toronto_venues.head()

(1708, 7)


,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,The Beaches,43.676357,-79.293031,Dip 'n Sip,43.678897,-79.297745,Coffee Shop


Let's check how many venues were returned for each neighborhood

In [166]:
neighborhoods_with_Toronto_venues.groupby('Neighbourhood').count()

,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,56,56,56,56,56,56
"Brockton,Exhibition Place,Parkdale Village",21,21,21,21,21,21
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",15,15,15,15,15,15
"Cabbagetown,St. James Town",46,46,46,46,46,46
Central Bay Street,85,85,85,85,85,85
"Chinatown,Grange Park,Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


#### Let's find out how many unique categories can be curated from all the returned venues

In [167]:
print('There are {} uniques categories.'.format(len(neighborhoods_with_Toronto_venues['Venue Category'].unique())))

There are 240 uniques categories.


### Analyze Each Neighborhood

In [168]:
# one hot encoding
neighborhoods_with_Toronto_onehot = pd.get_dummies(neighborhoods_with_Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
neighborhoods_with_Toronto_onehot['Neighbourhood'] = neighborhoods_with_Toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [neighborhoods_with_Toronto_onehot.columns[-1]] + list(neighborhoods_with_Toronto_onehot.columns[:-1])
neighborhoods_with_Toronto_onehot = neighborhoods_with_Toronto_onehot[fixed_columns]

neighborhoods_with_Toronto_onehot.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

And let's examine the new dataframe size.

In [169]:
neighborhoods_with_Toronto_onehot.shape

(1708, 241)

#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [170]:
neighborhoods_with_Toronto_grouped = neighborhoods_with_Toronto_onehot.groupby('Neighbourhood').mean().reset_index()
neighborhoods_with_Toronto_grouped.head()

,Neighbourhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Workshop,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Baseball Stadium,Basketball Stadium,Beach,Bed & Breakfast,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Stop,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Arts Building,College Gym,College Rec Center,Colombian Restaurant,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Festival,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gastropub,Gay Bar,General Entertainment,General Travel,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Health Food Store,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hospital,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Korean Restaurant,Lake,Latin American Restaurant,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Movie Theater,Museum,Music Store,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Park,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Pizza Place,Plane,Playground,Plaza,Poke Place,Polish Restaurant,Portuguese Restaurant,Poutine Place,Pub,Ramen Restaurant,Record Shop,Recording Studio,Rental Car Location,Restaurant,Roof Deck,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Smoothie Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Strip Club,Supermarket,Supplement Shop,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide,King,Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.03,0.0,0.0,0.000000,0.01,0.0,0.03,0.0,0.0000,0.000000,0.0,0.000000,0.020000,0.0,0.040000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.01,0.000000,0.01,0.020000,0.0000,0.0,0.01,0.03,0.010000,0.0,0.0,0.0,0.050000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.010000,0.000000,0.080000,0.0,0.0,0.0,0.01,0.000000,0.0000,0.020000,0.000000,0.03,0.0,0.000000,0.0,0.0,0.01,0.01,0.01,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.01,0.0,0.0,0.0

#### Let's confirm the new size

In [171]:
neighborhoods_with_Toronto_grouped.shape

(38, 241)

#### Let's print each neighborhood along with the top 5 most common venues

In [172]:
num_top_venues = 5

for hood in neighborhoods_with_Toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = neighborhoods_with_Toronto_grouped[neighborhoods_with_Toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
              venue  freq
0       Coffee Shop  0.08
1              Café  0.05
2        Steakhouse  0.04
3               Bar  0.04
4  Asian Restaurant  0.03


----Berczy Park----
          venue  freq
0   Coffee Shop  0.09
1  Cocktail Bar  0.05
2          Café  0.04
3   Cheese Shop  0.04
4      Beer Bar  0.04


----Brockton,Exhibition Place,Parkdale Village----
            venue  freq
0  Breakfast Spot  0.10
1     Coffee Shop  0.10
2            Café  0.10
3             Gym  0.05
4   Grocery Store  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                  venue  freq
0           Yoga Studio  0.06
1  Gym / Fitness Center  0.06
2               Brewery  0.06
3        Farmers Market  0.06
4            Restaurant  0.06


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
              venue  freq
0   Airport Service  0.13
1  Airport Terminal  0.13
2          Boutique  0.07
3      

#### Let's put that into a *pandas* dataframe

First, let's write a function to sort the venues in descending order.

In [173]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [174]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = neighborhoods_with_Toronto_grouped['Neighbourhood']

for ind in np.arange(neighborhoods_with_Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(neighborhoods_with_Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Bar,Steakhouse,American Restaurant,Asian Restaurant,Burger Joint,Thai Restaurant,Hotel,Cosmetics Shop
1,Berczy Park,Coffee Shop,Cocktail Bar,Farmers Market,Seafood Restaurant,Bakery,Steakhouse,Café,Cheese Shop,Beer Bar,Jazz Club
2,"Brockton,Exhibition Place,Parkdale Village",Café,Breakfast Spot,Coffee Shop,Convenience Store,Climbing Gym,Burrito Place,Stadium,Restaurant,Caribbean Restaurant,Bar
3,Business Reply Mail Processing Centre 969 Eastern,Yoga Studio,Auto Workshop,Garden Center,Light Rail Station,Fast Food Restaurant,Farmers Market,Comic Shop,Park,Recording Studio,Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Airport Terminal,Airport Service,Plane,Coffee Shop,Boat or Ferry,Sculpture Garden,Harbor / Marina,Boutique,Airport Lounge,Airport Gate


### Cluster Neighborhoods

Run *k*-means to cluster the neighborhood into 5 clusters.

Import Library

In [175]:
from sklearn.cluster import KMeans

In [176]:
# set number of clusters
kclusters = 5

neighborhoods_with_Toronto_grouped_clustering = neighborhoods_with_Toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(neighborhoods_with_Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [177]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

neighborhoods_with_Toronto_merged = neighborhoods_with_Toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
neighborhoods_with_Toronto_merged = neighborhoods_with_Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')

neighborhoods_with_Toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,2,Health Food Store,Neighborhood,Coffee Shop,Trail,Pub,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop
1,M4K,East Toronto,"The Danforth West,Riverdale",43.679557,-79.352188,2,Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Pub,Bookstore,Brewery,Bubble Tea Shop,Burger Joint
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.668999,-79.315572,2,Pizza Place,Sandwich Place,Pet Store,Brewery,Food & Drink Shop,Steakhouse,Fish & Chips Shop,Light Rail Station,Board Shop,Liquor Store
3,M4M,East Toronto,Studio District,43.659526,-79.340923,2,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space


Finally, let's visualize the resulting clusters

In [178]:
# Import Libraries
import matplotlib.cm as cm
import matplotlib.colors as colors

In [179]:
# create map
map_clusters = folium.Map(location=[latitude_toronto, longitude_toronto], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(neighborhoods_with_Toronto_merged['Latitude'], neighborhoods_with_Toronto_merged['Longitude'], neighborhoods_with_Toronto_merged['Neighbourhood'], neighborhoods_with_Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine Clusters

Now, you can examine each cluster and determine the discriminating venue categories that distinguish each cluster.

#### Cluster 1

In [180]:
neighborhoods_with_Toronto_merged.loc[neighborhoods_with_Toronto_merged['Cluster Labels'] == 0, neighborhoods_with_Toronto_merged.columns[[2] + list(range(6, neighborhoods_with_Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Lawrence Park,Park,Swim School,Bus Line,Yoga Studio,Dog Run,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
10,Rosedale,Park,Playground,Trail,Building,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop,Dumpling Restaurant


#### Cluster 2

In [181]:
neighborhoods_with_Toronto_merged.loc[neighborhoods_with_Toronto_merged['Cluster Labels'] == 1, neighborhoods_with_Toronto_merged.columns[[2] + list(range(6, neighborhoods_with_Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,"Moore Park,Summerhill East",Playground,Colombian Restaurant,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant


#### Cluster 3

In [182]:
neighborhoods_with_Toronto_merged.loc[neighborhoods_with_Toronto_merged['Cluster Labels'] == 2, neighborhoods_with_Toronto_merged.columns[[2] + list(range(6, neighborhoods_with_Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,The Beaches,Health Food Store,Neighborhood,Coffee Shop,Trail,Pub,Yoga Studio,Eastern European Restaurant,Dog Run,Doner Restaurant,Donut Shop
1,"The Danforth West,Riverdale",Greek Restaurant,Coffee Shop,Italian Restaurant,Furniture / Home Store,Ice Cream Shop,Pub,Bookstore,Brewery,Bubble Tea Shop,Burger Joint
2,"The Beaches West,India Bazaar",Pizza Place,Sandwich Place,Pet Store,Brewery,Food & Drink Shop,Steakhouse,Fish & Chips Shop,Light Rail Station,Board Shop,Liquor Store
3,Studio District,Café,Coffee Shop,Italian Restaurant,Bakery,American Restaurant,Yoga Studio,Comfort Food Restaurant,Seafood Restaurant,Sandwich Place,Cheese Shop
5,Davisville North,Clothing Store,Food & Drink Shop,Sandwich Place,Breakfast Spot,Gym,Park,Hotel,Dance Studio,Donut Shop,Dumpling Restaurant
6,North Toronto West,Sporting Goods Shop,Clothing Store,Coffee Shop,Mexican Restaurant,Rental Car Location,Café,Bagel Shop,Diner,Dessert Shop,Salon / Barbershop
7,Davisville,Pizza Place,Sandwich Place,Dessert Shop,Gym,Italian Restaurant,Restaurant,Café,Coffee Shop,Sushi Restaurant,Pub
9,"Deer Park,Forest Hill SE,Rathnelly,South Hill,...",Coffee Shop,Pub,Supermarket,Sports Bar,Liquor Store,Vietnamese Restaurant,Bagel Shop,Light Rail Station,Pizza Place,Sushi Restaurant
11,"Cabbagetown,St. James Town",Restaurant,Coffee Shop,Park,Grocery Store,Bakery,Italian Restaurant,Pizza Place,Café,Chinese Restaurant,Pub
12,Church and Wellesley,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Restaurant,Gay Bar,Gym,Bubble Tea Shop,Burger Joint,Fast Food Restaurant,Café


#### Cluster 4

In [183]:
neighborhoods_with_Toronto_merged.loc[neighborhoods_with_Toronto_merged['Cluster Labels'] == 3, neighborhoods_with_Toronto_merged.columns[[2] + list(range(6, neighborhoods_with_Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Roselawn,Home Service,Garden,Yoga Studio,Discount Store,Festival,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space,Ethiopian Restaurant


#### Cluster 5

In [184]:
neighborhoods_with_Toronto_merged.loc[neighborhoods_with_Toronto_merged['Cluster Labels'] == 4, neighborhoods_with_Toronto_merged.columns[[2] + list(range(6, neighborhoods_with_Toronto_merged.shape[1]))]]

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
23,"Forest Hill North,Forest Hill West",Mexican Restaurant,Trail,Jewelry Store,Sushi Restaurant,Yoga Studio,Dog Run,Fast Food Restaurant,Farmers Market,Falafel Restaurant,Event Space
